In [29]:
from tensorflow.keras.models import load_model

# Returns a compiled model identical to the previous one
model_path = 'piano_mfccs_RNN-LSTM.keras'
model = load_model(model_path)

In [47]:
# load mp3 and convert to amp spectrogram
import librosa
import numpy as np

mp3_file_path = 'piano_theme1.mp3'
y, sr = librosa.load(mp3_file_path)
# Compute MFCCs
n_mfcc=13
hop_length=512
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)

In [48]:
mfccs.shape

(13, 5178)

In [49]:
# Transpose the mfccs to have time dimension first for splitting
mfccs_t = np.transpose(mfccs)

# Calculate the number of chunks
num_chunks = mfccs_t.shape[0] // 50

# Split the transposed mfccs into chunks
chunks = np.array_split(mfccs_t[:num_chunks*50], num_chunks)

# If there are remaining frames, pad the last chunk and add to chunks
if mfccs_t.shape[0] % 50 != 0:
    last_chunk = np.pad(mfccs_t[num_chunks*50:], ((0, 50 - mfccs_t.shape[0] % 50), (0, 0)), mode='constant')
    chunks.append(last_chunk)

# Transpose back the chunks
X = np.array([np.transpose(chunk) for chunk in chunks])

In [50]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
for i, note in enumerate(X):
    X[i] = scaler.fit_transform(note) # transform the whole thing

In [51]:
predictions = model.predict(X)
rounded_predictions = np.round(predictions).astype(int)
predictions_list = rounded_predictions.tolist()

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [52]:
pred_notes=[]
for note in predictions_list:
    pred_notes.append(note[0])

In [53]:
import mido
from mido import Message, MidiFile, MidiTrack

# Assuming 'predictions_list' is your list of integers
# predictions_list = ...

# Create a new MIDI file and a track
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

tempo = mido.bpm2tempo(120)  # Set tempo to 120 beats per minute
track.append(mido.MetaMessage('set_tempo', tempo=tempo, time=0))

# Add notes to the track
time = 0
for note in pred_notes:
    # Note on
    track.append(Message('note_on', note=note, velocity=64, time=time))
    # Note off after 1 second (480 ticks)
    track.append(Message('note_off', note=note, velocity=64, time=480))
    # Add delay before next note
    time = 240

# Save the MIDI file
mid.save('output.mid')